In [1]:
# Imports
import sys
import sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import manual_variable_initialization

import numpy as np
import os

import pickle

print('TensorFlow version: {0}'.format(tf.__version__))

TensorFlow version: 2.0.0


In [2]:
save_files_location = '/home/idies/workspace/Storage/abhimat/persistent/TEP_Bot/'

In [3]:
# # Seeding

# np.random.seed(42)
# tf.random.set_seed(42)

In [4]:
# Pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Loading the text data

In [5]:
filepath = save_files_location + 'TEP_talks.txt'
with open(filepath) as f:
    TEP_text = f.read()

In [6]:
print(TEP_text[:500])

Title: Bootstrapping Inflationary Fluctuations
Abstract: In flat space, four point scattering amplitudes at weak coupling can be fully determined from Lorentz symmetry, unitarity and causality. The resulting scattering amplitude depends on model details only through coupling constants and the particle content of the theory. I will show how the analogous story works in the case of inflationary fluctuations. We found explicit expressions for inflationary three and four-point functions, whose shape


In [7]:
''.join(sorted(set(TEP_text)))

'\n !"$%&\'()+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_abcdefghijklmnopqrstuvwxyz{}~×éöˆ̈ℓ∗∼≤'

## Construct dataset

In [8]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(TEP_text)

In [9]:
tokenizer.texts_to_sequences(['Abstract:'])

[[32, 20, 8, 3, 9, 5, 12, 3, 31]]

In [10]:
tokenizer.sequences_to_texts([[32, 20, 8, 3, 9, 5, 12, 3, 31]])

['A b s t r a c t :']

In [11]:
max_id = len(tokenizer.word_index) # Number of distinct characters
dataset_size = tokenizer.document_count # Total number of characters

print('max_id = {0}'.format(max_id))
print('dataset_size = {0}'.format(dataset_size))

max_id = 100
dataset_size = 136477


In [12]:
[encoded] = np.array(tokenizer.texts_to_sequences([TEP_text])) - 1
train_size = dataset_size * 100 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## Chop dataset into windows (for Stateful RNN)

In [13]:
# Chop dataset into windows

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead

dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)

In [14]:
# Flatten windows
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [15]:
# Batch, and no shuffling

# dataset = dataset.shuffle(10000).batch(batch_size)
# dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.prefetch(1)

In [16]:
batch_size = 16

encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)

dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
# dataset = dataset.map(
#     lambda X_batch, Y_batch: (embedding(X_batch), Y_batch))
dataset = dataset.prefetch(1)

## Char-RNN Model

In [17]:
num_oov_buckets = 0

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim = max_id + num_oov_buckets,
                           batch_input_shape=[batch_size, None],
                           output_dim = 2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, 2]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation='softmax'))
])

# Stateful RNNs: dropouts don't work (https://github.com/ageron/handson-ml2/issues/32)

In [18]:
model_load_file = save_files_location + '/TEP_Bot_Save_2020-05-07_stateful.h5'
model_load = keras.models.load_model(model_load_file)

model.set_weights(model_load.get_weights())

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [19]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [21]:
# model_stateless = keras.models.load_model('TEP_Bot_Save_2020-05-05.h5')
# print('Weights:')
# print(model_stateless.get_weights())

# model = tf.keras.models.load_model('TEP_Bot_Save_2020-05-03.h5')

weights_file_load = save_files_location + 'TEP_Bot_2020-05-07_weights.pkl'

# with open(weights_file_load, 'rb') as in_pickle:
#     weights = pickle.load(in_pickle)

# model.set_weights(weights)

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

save_model_file = save_files_location + 'TEP_Bot_Save_2020-05-07_stateful.h5'

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_file,
                                                 verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 2)             200       
_________________________________________________________________
gru (GRU)                    (16, None, 128)           50688     
_________________________________________________________________
gru_1 (GRU)                  (16, None, 128)           99072     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 100)         12900     
Total params: 162,860
Trainable params: 162,860
Non-trainable params: 0
_________________________________________________________________


In [ ]:

history = model.fit(dataset,
                    steps_per_epoch = train_size // batch_size // n_steps,
                    epochs=50,
                    callbacks = [cp_callback, ResetStatesCallback()]
                   )

model.save(save_model_file)

Train for 85 steps
Epoch 1/50
84/85 [============================>.] - ETA: 2s - loss: 1.2818
Epoch 00001: saving model to /home/idies/workspace/Storage/abhimat/persistent/TEP_Bot/TEP_Bot_Save_2020-05-07_stateful.h5
85/85 [==============================] - 187s 2s/step - loss: 1.2820
Epoch 2/50
84/85 [============================>.] - ETA: 2s - loss: 1.2656
Epoch 00002: saving model to /home/idies/workspace/Storage/abhimat/persistent/TEP_Bot/TEP_Bot_Save_2020-05-07_stateful.h5
85/85 [==============================] - 209s 2s/step - loss: 1.2658
Epoch 3/50
84/85 [============================>.] - ETA: 2s - loss: 1.2514
Epoch 00003: saving model to /home/idies/workspace/Storage/abhimat/persistent/TEP_Bot/TEP_Bot_Save_2020-05-07_stateful.h5
85/85 [==============================] - 236s 3s/step - loss: 1.2516
Epoch 4/50
84/85 [============================>.] - ETA: 2s - loss: 1.2382
Epoch 00004: saving model to /home/idies/workspace/Storage/abhimat/persistent/TEP_Bot/TEP_Bot_Save_2020-05-0

In [ ]:
weights = model.get_weights()

weights_file_save = save_files_location + 'TEP_Bot_2020-05-07_stateful_weights.pkl'

with open(weights_file_save, 'wb') as out_pickle:
    pickle.dump(weights, out_pickle)

In [ ]:
model.summary()

## Evaluating the model

In [ ]:
# model.load('TEP_Bot_Save_2020-05-02.h5')
# with open('history_2020-05-03.pkl', 'wb') as in_pickle:
#     history = pickle.load(in_pickle)

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left');

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left');

## Generating text from the model

In [ ]:
print('Ready to go')

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return embedding(X_batch)

In [ ]:
X_new = preprocess(["In this tal"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] 